In [11]:
from glob import glob
import soundfile as sf

timestamps = [i * 0.02 for i in range(1500 + 1)]

files = glob('audio_16k/*')
len(files)

21024

In [10]:
labels = set()


for f in files:
    l = f.split('_')[-1].replace('.wav', '')
    labels.add(l)
    
labels

{'cough', 'laughter', 'sigh', 'sneeze', 'sniff', 'throatclearing'}

In [21]:
import soundfile as sf
from tqdm import tqdm

replace = {
    'laughter': 'laugh'
}

langs = ['en', 'ms']
tasks = ['transcribenonverbal']

data = []
for f in tqdm(files):
    y, sr = sf.read(f)
    label = f.split('_')[-1].replace('.wav', '')
    label = replace.get(label, label)
    label = f'<|{label}|>'
    
    start = min(timestamps, key=lambda t: abs(t - 0.0))
    end = min(timestamps, key=lambda t: abs(t - (len(y) / sr)))

    for l in langs:
        for t in tasks:
            text = f'<|startoftranscript|><|{l}|><|{t}|><|{start:.2f}|>{label}<|{end:.2f}|><|endoftext|>'
            data.append({
                'audio_filename': f,
                'new_text': text,
                'source': 'VocalSound'
            })
    
len(data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 21024/21024 [00:05<00:00, 3584.96it/s]


42048

In [22]:
data[-1]

{'audio_filename': 'audio_16k/m1170_0_throatclearing.wav',
 'new_text': '<|startoftranscript|><|ms|><|transcribenonverbal|><|0.00|><|throatclearing|><|2.22|><|endoftext|>',
 'source': 'VocalSound'}

In [23]:
import pandas as pd

pd.DataFrame(data).to_parquet('vocalsound.parquet')

In [24]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="vocalsound.parquet",
    path_in_repo="data/vocalsound-00000-of-00001.parquet",
    repo_id="mesolitica/Speech-Nonverbal-Whisper",
    repo_type="dataset",
)

vocalsound.parquet:   0%|          | 0.00/372k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Speech-Nonverbal-Whisper/commit/291fcbedc500c55ba7baf3c52f4ef179949424da', commit_message='Upload data/vocalsound-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='291fcbedc500c55ba7baf3c52f4ef179949424da', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Speech-Nonverbal-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Speech-Nonverbal-Whisper'), pr_revision=None, pr_num=None)